In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import os
#! apt-get update -qq
#! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
#os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin" + os.environ["PATH"]
#! java -version

#! pip install --ignore-installed pyspark==2.4.4
#! pip install --ignore-installed spark-nlp==2.6.2
#!pip install findspark

#!sudo apt install openjdk-8-jdk
#!sudo update-alternatives --config java

In [ ]:
!pip install pyspark
!pip install findspark
!pip install sparknlp
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --config java
!java -version

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 58.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=d577f60d24586ce33afe9e9daba0a45add1779602b99781bfef7cd5fc0c43fe1
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
     |████████████████████████████████| 91 kB 6.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho font

In [ ]:
#18521589 - Vo Dinh Tu  
#18521564 - Nguyen Huu Truong
#18520630 - Ho Anh Dung

In [ ]:
import pyspark
import pandas as pd
from IPython.display import display, clear_output
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as f
from pyspark.ml import PipelineModel
from pyspark.sql.functions import udf
from pyspark.sql.streaming import DataStreamReader
import html
from pyspark.sql.functions import col, when , regexp_replace


#Set up folder Spark

In [ ]:
import sparknlp 
spark = sparknlp.start()
 
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.1.1
Apache Spark version:  3.1.2


In [ ]:
spark = pyspark.sql.SparkSession.builder.master("local[4]").appName("PySpark").getOrCreate()

In [ ]:
%mkdir /content/CrawldataIMDB

#Load data

In [ ]:
IN_PATH='/content/CrawldataIMDB'
#timestampformat='EEE MMM dd HH:mm:ss zzzz yyyy'
spark.sql('set spark.sql.legacy.timeParserPolicy=LEGACY')
spark= SparkSession.builder.appName("StructuredStreamingExample").getOrCreate()
spark.conf.set('spark.sql.legacy.timeParserPolicy',"LEGACY")
schema=spark.read.json(IN_PATH).limit(10).schema
spark_reader=spark.readStream.schema(schema)

In [ ]:
#for data_imdb in IN_PATH:
streaming_data_raw=(spark_reader.json(IN_PATH).select(f.col('date').alias('timestamp'),f.col('reviewer_name').alias('user'),f.col('text').alias('review_detail'),).coalesce(1))
streaming_data_clean=streaming_data_raw
stream_writer=(streaming_data_raw.writeStream.queryName('data').trigger(once=True).outputMode('append').format('memory'))
query=stream_writer.start()

In [ ]:
display(spark.sql(f"SELECT * from {query.name}").show(5))

+-------------+------------------+--------------------+
|    timestamp|              user|       review_detail|
+-------------+------------------+--------------------+
| 3 April 2021|          ThomDerd|I am glad I watch...|
|26 March 2021|brycetulloch-13604|I liked the actio...|
| 4 April 2021|       movieliker1|This is another m...|
| 1 April 2021|          kenzibit|Rating basically ...|
| 2 April 2021|       alfredsmith|This is a mindles...|
+-------------+------------------+--------------------+
only showing top 5 rows



None

# Load our pretrained model using custom dataset

In [ ]:
sentiment_model=PipelineModel.load('/content/drive/MyDrive/BigData_Project/NLP_model_lr')

In [ ]:
raw_sentiment=sentiment_model.transform(streaming_data_clean)
sentiment=raw_sentiment.select('timestamp','user','review_detail','probability')

In [ ]:
from pyspark.ml.functions import vector_to_array
final = sentiment.select('timestamp','user','review_detail',"probability")\
.withColumn("probability",vector_to_array(f.col("probability")))\
.withColumn("probability",f.element_at(f.col("probability"),-1))
result = final.withColumn("sentiment",
                        f.when(f.col("probability") < 0.475,'negative')
                        .when((f.col("probability") >= 0.475) & (f.col("probability") <= 0.675),'neutral')
                        .when(f.col("probability") > 0.675,'positive')
                        .otherwise(f.col("probability")))

In [ ]:
result=result.select('timestamp','user','review_detail','sentiment')
sentiment_count_result=result.groupBy("sentiment").agg(f.count("sentiment").alias("count")).sort("count")

In [ ]:
stream_writer1=(result.writeStream.queryName('result').trigger(processingTime='5 seconds').outputMode('append').format('memory'))
query1=stream_writer1.start()
stream_writer2=(sentiment_count_result.writeStream.queryName('data').trigger(processingTime='5 seconds').outputMode('complete').format('memory'))
query2=stream_writer2.start()

# DEMO real-time sentiment anaysis on our custom dataset

In [ ]:
if raw_sentiment.isStreaming:
    from time import sleep
    for x in range(0,2000):
        try:
            if not query1.isActive:
                print('Query not active')
                break
            print('Showing live view refreshed every 5 seconds')
            print(f"Seconds passed: {x*5}")
            result1=spark.sql(f"SELECT * from {query1.name}")
            result2=spark.sql(f"SELECT * from {query2.name}")
            print(len(result1.toPandas()))
            display(result1.toPandas())
            display(result2.toPandas())
            sleep(2)
            clear_output(wait=True)
        except KeyboardInterrupt:
            print('break')
            break
    print('Live view ended...')
else:
    print("Not streaming")